# Analyzing Parade

As a board game enjoyer, I recently discovered an excellent card game called [Parade](https://boardgamegeek.com/boardgame/56692/parade). It is not that different from [Arboretum](https://boardgamegeek.com/boardgame/140934/arboretum), except it's more random so less cutthroat. Basically, you try to take as few cards as possible from the parade because cards score negative points. After playing a couple games, I came up with some hypothesis about the game's strategy and mechanics.

Unlike Arboretum, where the general strategy is to play small discard small. It is hard to come up with a strategy for Parade. It feels very random. I decided to simulate the game and see what I could come up with. I impelemented the simulator in Rust to gather some statistics using a 90%-there strategy. It's not perfect, but it's good enough to give some ideas about Parade's characteristics, especially just how much does the early game affects the game's outcome. I also wanted to optimize to avoid being forced to take cards from the parade, which is a bad experience. 

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.ticker import PercentFormatter
import itertools

The simulator takes different parameters. players count, number of suits, number of ranks, number of iterations, and strategies. The strategy is a function that takes a player's hand and returns a card to add to the parade. The simulator runs the game for a number of iterations and outputs the score of the game. Then in pandas, I calculate the statistics for configuration.

In [3]:
def run(players=2, suits=6, ranks=11, iters=50000, strats=[1]):
    result = []
    for s in itertools.product(strats, repeat=players):
        strat = " ".join(list(map(str,s)))
        ! cargo run --release -- --players=$players --suits=$suits --ranks=$ranks --iters=$iters $strat
        df = pd.read_csv("output.csv", header=None)
        df = df.set_axis(["score "+str(i) for i in range(players)]+ ["forced take "+str(i) for i in range(players)], axis=1)
        means = df.mean()
        for i, m in enumerate(s): 
            means[str(i)] = m
        result.append(means)
    return pd.DataFrame(result)

I wrote 3 strategies:

1. Always use the first card in hand.
2. For each card in hand, find the lowest sum of ranks from the ejected cards.
3. On top of 2, voluntarily take rank 0-2 cards from the parade.

In [4]:
df = run(players=2, suits=6, ranks=11, iters=5000, strats=[0, 1, 2])
df['scorediff'] = df['score 0'] - df['score 1']
df['forcedtakediff'] = df['forced take 0'] - df['forced take 1']
df

    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=6 --ranks=11 --iters=5000 0 0`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=6 --ranks=11 --iters=5000 0 1`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=6 --ranks=11 --iters=5000 0 2`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=6 --ranks=11 --iters=5000 1 0`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=6 --ranks=11 --iters=5000 1 1`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=6 --ranks=11 --iters=5000 1 2`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=6 --ranks=11 --iters=5000 2 0`
    Finished release [optim

,score 0,score 1,forced take 0,forced take 1,0,1,scorediff,forcedtakediff
0,80.8092,95.4862,3.3016,3.4592,0.0,0.0,-14.6770,-0.1576
1,146.9090,35.2912,6.9034,12.0610,0.0,1.0,111.6178,-5.1576
2,145.3782,36.1774,6.8086,11.7340,0.0,2.0,109.2008,-4.9254
3,26.9696,161.1628,11.7688,7.1186,1.0,0.0,-134.1932,4.6502
4,71.0302,88.1068,16.3072,16.6150,1.0,1.0,-17.0766,-0.3078
5,70.0976,88.9962,16.2268,16.4954,1.0,2.0,-18.8986,-0.2686
6,27.6018,160.3182,11.4250,7.0140,2.0,0.0,-132.7164,4.4110
7,71.4762,87.6924,16.1492,16.5404,2.0,1.0,-16.2162,-0.3912
8,70.7912,88.4290,16.1066,16.4522,2.0,2.0,-17.6378,-0.3456


In [5]:
df.pivot_table(index="0", columns="1", values='scorediff', fill_value=0)

1,0.0,1.0,2.0
0,,,
0.0,-14.6770,111.6178,109.2008
1.0,-134.1932,-17.0766,-18.8986
2.0,-132.7164,-16.2162,-17.6378


In [6]:
df.pivot_table(index="0", columns="1", values='forcedtakediff', fill_value=0)

1,0.0,1.0,2.0
0,,,
0.0,-0.1576,-5.1576,-4.9254
1.0,4.6502,-0.3078,-0.2686
2.0,4.4110,-0.3912,-0.3456


Next let's look at number of forced takes.

In [9]:
results = []
for i in range(4, 10):
    for j in range(5, 20):
        df = run(players=2, suits=i, ranks=j, iters=5000)
        df['score mean'] = df.loc[:, df.columns.str.startswith('score')].loc[0].mean()
        df['forced take mean'] = df.loc[:, df.columns.str.startswith('forced take')].loc[0].mean()
        df["suits"] = i
        df["ranks"] = j
        results.append(df)
df = pd.concat(results)
df

    Finished release [optimized] target(s) in 0.03s
     Running `target/release/parade --players=2 --suits=4 --ranks=5 --iters=5000 1 1`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=4 --ranks=6 --iters=5000 1 1`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=4 --ranks=7 --iters=5000 1 1`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=4 --ranks=8 --iters=5000 1 1`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=4 --ranks=9 --iters=5000 1 1`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=4 --ranks=10 --iters=5000 1 1`
    Finished release [optimized] target(s) in 0.02s
     Running `target/release/parade --players=2 --suits=4 --ranks=11 --iters=5000 1 1`
    Finished release [optimized]

,score 0,score 1,forced take 0,forced take 1,0,1,score mean,forced take mean,suits,ranks
0,1.8380,2.1316,1.4226,1.2672,1.0,1.0,1.9848,1.3449,4,5
0,4.9492,6.6726,2.7480,2.6828,1.0,1.0,5.8109,2.7154,4,6
0,9.6126,12.9730,3.9334,4.1814,1.0,1.0,11.2928,4.0574,4,7
0,15.2848,20.6574,5.1748,5.4690,1.0,1.0,17.9711,5.3219,4,8
0,22.6098,29.5540,6.3616,6.6466,1.0,1.0,26.0819,6.5041,4,9
...,...,...,...,...,...,...,...,...,...,...
0,258.6592,275.0456,38.5822,38.7984,1.0,1.0,266.8524,38.6903,9,15
0,268.1056,317.6576,40.7646,41.0216,1.0,1.0,292.8816,40.8931,9,16
0,330.7328,358.7924,43.2768,43.8000,1.0,1.0,344.7626,43.5384,9,17
0,345.7052,404.9486,45.3104,45.6418,1.0,1.0,375.3269,45.4761,9,18


In [13]:
pd.pivot_table(df, index="suits", columns="ranks", values='forced take mean', fill_value=0)

ranks,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
suits,,,,,,,,,,,,,,,
4,1.3449,2.7154,4.0574,5.3219,6.5041,7.6202,8.6429,9.6492,10.5620,11.4604,12.3244,13.1466,13.9855,14.7990,15.5209
5,3.2315,4.7147,6.7806,8.0767,10.0745,11.1845,12.9405,13.9629,15.5466,16.3747,17.8457,18.6388,20.0670,20.7876,22.0913
6,4.3665,6.5587,8.5980,10.6677,12.6868,14.6682,16.4611,18.2093,19.8362,21.3696,22.7886,24.2115,25.6451,26.9576,28.2954
7,6.1512,8.3063,11.0536,13.1166,15.8994,17.9168,20.5526,22.3319,24.6707,26.2480,28.3931,29.8270,31.8004,33.1161,35.0500
8,7.1593,9.9771,12.7290,15.4548,18.2778,20.9966,23.6924,26.3016,28.7580,31.0237,33.2301,35.3722,37.4398,39.3331,41.2711
9,8.8140,11.6069,15.0119,17.7703,21.2190,24.0149,27.4443,30.1499,33.3061,35.7630,38.6903,40.8931,43.5384,45.4761,48.0263


So if you really hate being forced to take a card from the parade, you can playing with 4 suits and 5 ranks.